In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [5]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
# train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.75it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [8]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df in [df_hist, df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['hour'] = df['purchase_date'].dt.hour
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)

In [6]:
df_hist.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
df_new.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)

In [7]:
df_hist.head()

,card_id,purchase_date,authorized_flag,month_lag,state_id,subsector_id,first_active_month,all_term,yyyymm,yyyymmdd,weekofyear,yyyy_week,purchase_amount_over
0,C_ID_4e6213e9bc,2017-06-25 15:33:07,1,-8,16,37,2016-06-01,699,2017-06,2017-06-25,25,2017-25,0.0
1,C_ID_4e6213e9bc,2017-07-15 12:10:45,1,-7,16,16,2016-06-01,699,2017-07,2017-07-15,28,2017-28,0.0
2,C_ID_4e6213e9bc,2017-08-09 22:04:29,1,-6,16,37,2016-06-01,699,2017-08,2017-08-09,32,2017-32,0.0
3,C_ID_4e6213e9bc,2017-09-02 10:06:26,1,-5,16,34,2016-06-01,699,2017-09,2017-09-02,35,2017-35,0.0
4,C_ID_4e6213e9bc,2017-03-10 01:14:19,1,-11,16,37,2016-06-01,699,2017-03,2017-03-10,10,2017-10,0.0


### 日付は関係なく、直近何回、初回から何回の購買におけるデータを集計する
first 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する  
last 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する  
同日購買も重複削除せずに集計するversion

In [11]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
aggs = {}
aggs['date_diff'] = ['mean', 'std']


df_list = [df_new, df_hist]
fname_list = ['new', 'hist']


for df, hist_new_name in zip(df_list, fname_list):

    df = df[[key, 'purchase_date', 'date_diff', 'month_diff', 'yyyymm']]

    for i in range(5, 26, 5):
        
        df.sort_values(by=[key, 'purchase_date'], inplace=True)
        df = utils.row_number(df=df, level=key)
    
        feat_n = f'{hist_new_name}_first'
        first = df.query(f"row_no<={i}")
        tmp = first.groupby(key)['date_diff'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
        df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
        
        
    for i in range(5, 26, 5):
        
        df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
        df = utils.row_number(df=df, level=key)
        
        feat_n = f'{hist_new_name}_last'
        last = df.query(f"row_no<={i}")
        tmp = last.groupby(key)['date_diff'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
        df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
    
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 187.72it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_first5_date_diff_mean,new_first5_date_diff_std,new_first10_date_diff_mean,new_first10_date_diff_std,new_first15_date_diff_mean,new_first15_date_diff_std,new_first20_date_diff_mean,new_first20_date_diff_std,new_first25_date_diff_mean,new_first25_date_diff_std,new_first30_date_diff_mean,new_first30_date_diff_std,new_last5_date_diff_mean,new_last5_date_diff_std,new_last10_date_diff_mean,new_last10_date_diff_std,new_last15_date_diff_mean,new_last15_date_diff_std,new_last20_date_diff_mean,new_last20_date_diff_std,new_last25_date_diff_mean,new_last25_date_diff_std,new_last30_date_diff_mean,new_last30_date_diff_std,hist_first5_date_diff_mean,hist_first5_date_diff_std,hist_first10_date_diff_mean,hist_first10_date_diff_std,hist_first15_date_diff_mean,hist_first15_date_diff_std,hist_first20_date_diff_mean,hist_first20_date_diff_std,hist_first25_date_diff_mean,hist_first25_date_diff_std,hist_first30_date_diff_mean,hist_first30_date_diff_std,hist_last5_date_diff_mean,hist_last5_date_diff_std,hist_last10_date_diff_mean,hist_last10_date_diff_std,hist_last15_date_diff_mean,hist_last15_date_diff_std,hist_last20_date_diff_mean,hist_last20_date_diff_std,hist_last25_date_diff_mean,hist_last25_date_diff_std,hist_last30_date_diff_mean,hist_last30_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,316.8,3.271085,309.400000,9.215928,303.266667,11.798709,297.800000,14.192659,294.043478,16.576210,294.043478,16.576210,272.2,5.761944,279.000000,8.366600,284.200000,10.310882,290.300000,14.308996,294.043478,16.576210,294.043478,16.576210,568.8,1.483240,566.9,2.330951,565.466667,2.825058,564.20,3.381062,562.72,4.277071,561.400000,4.924254,331.4,2.509980,333.8,3.084009,336.200000,4.491261,338.60,5.807436,341.32,7.712112,344.300000,9.787924
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,332.0,24.238399,326.000000,26.191602,326.000000,26.191602,326.000000,26.191602,326.000000,26.191602,326.000000,26.191602,320.8,25.587106,326.000000,26.191602,326.000000,26.191602,326.000000,26.191602,326.000000,26.191602,326.000000,26.191602,741.2,1.788854,738.0,4.242641,735.333333,5.177791,733.15,5.993637,729.04,10.191827,725.100000,12.930451,354.2,1.303840,355.4,1.577621,356.000000,1.558387,356.65,1.814416,358.08,3.426855,359.633333,4.723261
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,266.0,NaN,266.000000,NaN,266.000000,NaN,266.000000,NaN,266.000000,NaN,266.000000,NaN,266.0,NaN,266.000000,NaN,266.000000,NaN,266.000000,NaN,266.000000,NaN,266.000000,NaN,731.6,8.384510,718.1,16.549253,707.600000,20.862817,693.00,32.266978,680.36,38.758741,663.900000,52.747512,379.6,47.992708,419.5,53.146025,455.866667,69.032980,494.85,91.297244,523.08,99.605020,546.766667,105.589538
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,295.2,14.889594,290.142857,14.915636,290.142857,14.915636,290.142857,14.915636,290.142857,14.915636,290.142857,14.915636,282.4,5.941380,290.142857,14.915636,290.142857,14.915636,290.142857,14.915636,290.142857,14.915636,290.142857,14.915636,479.0,0.707107,478.2,1.032796,476.733333,2.548576,475.10,3.768289,472.88,5.688585,470.566667,7.389104,329.8,2.863564,333.4,5.211099,337.466667,7.356889,344.35,15.062239,352.88,22.115831,363.100000,30.847064
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,320.8,3.898718,315.800000,6.014797,310.200000,9.879271,305.650000,11.766478,301.400000,13.622897,297.666667,15.040940,270.8,4.266146,274.300000,4.900113,277.266667,5.945787,280.600000,7.929957,283.800000,9.716824,288.066667,13.156704,432.4,1.673320,430.6,2.221111,427.866667,4.453998,425.55,5.633313,423.32,6.799020,421.300000,7.742182,325.4,0.547723,325.9,0.875595,327.000000,1.851640,329.00,4.103913,331.56,6.397135,334.100000,8.230934


In [12]:
fname = '130_ddf'
ignore_features = ['first_active_month', 'card_id']

df_feat = pd.concat([df_train, df_test], axis=0)
df_feat.fillna(-1, inplace=True)

for col in df_feat.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('date_diff')):continue
#     if not(col.count('trans')):continue

    feature = df_feat[col].values.astype('float32')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_all_{col}@', obj=feature)

### 132 date_diffの前半、後半における違いを特徴にする. 全体verも

In [16]:
from dateutil.parser import parse

# 日付の重複を削除するため
df_hist['purchase_date'] =  df_hist['purchase_date'].map(lambda x: str(x)[:10])
df_new['purchase_date'] =  df_new['purchase_date'].map(lambda x: str(x)[:10])

# 前回購買日との差分
def diff_shift_date(df, lag):
    
    # shiftは正が下方向、負が上方向にズレる
    df = df[[key, 'purchase_date']].drop_duplicates()
    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['shift1_date'] = df.groupby(key)['purchase_date'].shift(lag)
    df = df[[key, 'purchase_date', 'shift1_date']].dropna()
    df['date_diff_lag1'] = (df['purchase_date'] - df['shift1_date']).dt.days
    return df
    
for i in tqdm(range(1, 2, 1)):
    diff_hist = diff_shift_date(df_hist, i)
    diff_new = diff_shift_date(df_new, i)
    
display(diff_hist.head())
display(diff_new.head())

100%|██████████| 1/1 [00:22<00:00, 22.83s/it]


,card_id,purchase_date,shift1_date,date_diff
19095845,C_ID_00007093c1,2017-02-16,2017-02-14,2
19095866,C_ID_00007093c1,2017-02-20,2017-02-16,4
19095808,C_ID_00007093c1,2017-03-03,2017-02-20,11
19095905,C_ID_00007093c1,2017-03-06,2017-03-03,3
19095780,C_ID_00007093c1,2017-03-08,2017-03-06,2


,card_id,purchase_date,shift1_date,date_diff
1,C_ID_00007093c1,2018-04-09,2018-04-03,6
3,C_ID_0001238066,2018-03-03,2018-03-01,2
4,C_ID_0001238066,2018-03-04,2018-03-03,1
5,C_ID_0001238066,2018-03-10,2018-03-04,6
6,C_ID_0001238066,2018-03-16,2018-03-10,6


In [17]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

for diff, hist_new_name in zip([diff_hist, diff_new], ['hist', 'new']):
    df = diff[[key, 'purchase_date', 'date_diff_lag1']].drop_duplicates()
    
    aggs = {}
    aggs['date_diff_lag1'] = ['mean', 'std']

    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    df = utils.row_number(df=df, level=key)
    
    for i in range(5, 26, 5):
        
        feat_n = f'{hist_new_name}_first'
        first = df.query(f"row_no<={i}")
        tmp = first.groupby(key)['date_diff_lag1'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
        df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
    
    
    df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
    df = utils.row_number(df=df, level=key)
    for i in range(5, 26, 5):
         
        feat_n = f'{hist_new_name}_last'
        last = df.query(f"row_no<={i}")
        tmp = last.groupby(key)['date_diff_lag1'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
        df_test = df_test.merge(tmp.reset_index(), how='left', on=key)

100%|██████████| 3/3 [00:00<00:00, 180.74it/s]


In [18]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_first5_date_diff_mean,hist_first5_date_diff_std,hist_first10_date_diff_mean,hist_first10_date_diff_std,hist_first15_date_diff_mean,hist_first15_date_diff_std,hist_first20_date_diff_mean,hist_first20_date_diff_std,hist_first25_date_diff_mean,hist_first25_date_diff_std,hist_last5_date_diff_mean,hist_last5_date_diff_std,hist_last10_date_diff_mean,hist_last10_date_diff_std,hist_last15_date_diff_mean,hist_last15_date_diff_std,hist_last20_date_diff_mean,hist_last20_date_diff_std,hist_last25_date_diff_mean,hist_last25_date_diff_std,new_first5_date_diff_mean,new_first5_date_diff_std,new_first10_date_diff_mean,new_first10_date_diff_std,new_first15_date_diff_mean,new_first15_date_diff_std,new_first20_date_diff_mean,new_first20_date_diff_std,new_first25_date_diff_mean,new_first25_date_diff_std,new_last5_date_diff_mean,new_last5_date_diff_std,new_last10_date_diff_mean,new_last10_date_diff_std,new_last15_date_diff_mean,new_last15_date_diff_std,new_last20_date_diff_mean,new_last20_date_diff_std,new_last25_date_diff_mean,new_last25_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,1.4,0.547723,1.3,0.483046,1.200000,0.414039,1.20,0.410391,1.20,0.408248,1.8,1.095445,1.8,1.135292,1.866667,0.990430,2.00,1.076055,1.88,1.013246,2.0,1.00000,2.5,2.460804,2.6,2.292846,2.75,2.359193,2.75,2.359193,3.2,2.774887,3.0,2.357023,3.000000,2.645751,2.75,2.359193,2.75,2.359193
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,2.0,1.224745,2.2,1.032796,2.600000,1.404076,2.50,1.277333,2.84,2.823709,1.2,0.447214,1.2,0.421637,1.200000,0.414039,1.15,0.366348,1.20,0.500000,19.0,21.00000,19.0,21.000000,19.0,21.000000,19.00,21.000000,19.00,21.000000,19.0,21.000000,19.0,21.000000,19.000000,21.000000,19.00,21.000000,19.00,21.000000
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,7.0,7.968689,6.4,5.541761,7.200000,6.784436,9.55,12.816416,10.92,12.734206,24.4,22.534418,17.8,18.671428,18.666667,19.047560,15.60,17.409616,14.44,16.057397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,1.6,0.894427,1.4,0.699206,1.466667,0.743223,1.70,1.218282,2.36,2.395830,3.0,1.581139,2.8,1.316561,3.800000,3.629246,4.35,3.674593,4.80,3.708099,8.2,5.80517,7.0,5.966574,7.0,5.966574,7.00,5.966574,7.00,5.966574,5.0,3.807887,7.0,5.966574,7.000000,5.966574,7.00,5.966574,7.00,5.966574
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,1.0,0.000000,1.4,0.966092,1.333333,0.816497,1.55,1.099043,1.60,1.000000,1.0,0.000000,1.3,0.674949,1.466667,0.743223,1.40,0.680557,1.40,0.645497,2.6,3.04959,2.8,2.250926,2.4,2.028370,2.15,1.814416,2.24,1.738774,2.4,1.673320,2.0,1.247219,1.866667,1.245946,2.10,1.372665,2.20,1.755942


In [21]:
fname = '131_ddf'
ignore_features = ['first_active_month', 'card_id']

df_feat = pd.concat([df_train, df_test], axis=0)
df_feat.fillna(-1, inplace=True)

for col in df_feat.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('date_diff')):continue
#     if not(col.count('trans')):continue

    feature = df_feat[col].values.astype('float32')
    col = col.replace('date_diff', 'date_diff_lag1')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_all_{col}@', obj=feature)

In [54]:
df = diff[[key, 'purchase_date', 'date_diff']].drop_duplicates()
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
hist_new_name = 'new'
aggs = {}
aggs['date_diff'] = ['mean', 'std']

df.sort_values(by=[key, 'purchase_date'], inplace=True)
df = utils.row_number(df=df, level=key)
tmp = df.groupby(key)['date_diff'].agg(aggs)
new_cols = get_new_columns(f'{hist_new_name}', aggs)
tmp.columns = new_cols
df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
df_train.head()


100%|██████████| 3/3 [00:00<00:00, 160.55it/s]

,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_date_diff_mean,new_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,2.750000,2.359193
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,19.000000,21.000000
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,7.000000,5.966574
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,2.192308,1.720912
